# Band Gap Prediction for Nitride-Based Solid-State Electrolytes Using Machine Learning

In [11]:
import pandas as pd
import numpy as np
from mp_api.client import MPRester
print("All libraries imported successfully.")

# Replace with your API key
# Get from https://materialsproject.org/open
API_KEY = "UWFBjghRwpm3Rk7ytx3G4uvRGGz6OG5R"

All libraries imported successfully.


MPRester uses the latest mp-api client (2025 documentation) to query the materials.summary endpoint.

- **Targeted Filtering**: Uses specific filters (`elements=["N"]`, `exclude_elements=["S", "O"]`) to focus on pure nitrides, excluding oxynitrides and thionitrides.
- **Stability Consideration**: Limits `energy_above_hull` to ≤0.15 eV for stable or near-stable compounds, suitable for practical electrolyte applications.
- **Data Conversion**: Converts query results to a pandas DataFrame and saves to CSV for reproducibility and downstream analysis.

Output: csv with ~3,000 entries

In [13]:
def get_electrolyte_data():
    with MPRester(API_KEY) as mpr:
        # Filters for nitride-based electrolytes
        elements = ["N"]            # Must include nitrogen and lithium
        exlude_elements = ["S", "O"]      # Filter out oxynitrides and thionitrides
        num_elements = (2, 6)            # 2–6 elements for simple and complex nitrides

        # Required features
        fields = [
            "material_id",
            "formula_pretty",
            "band_gap",
            "volume",
            "cbm",                    # Conduction band minimum
            "density",
            "density_atomic",
            "efermi",
            "energy_above_hull",
            "num_magnetic_sites",
            "num_unique_magnetic_sites"
        ]

        # Query for nitride materials
        try:
            print("Querying Materials Project with filters:")
            docs = mpr.materials.summary.search(
                elements=elements,
                exclude_elements=exlude_elements,
                num_elements=num_elements,
                energy_above_hull=(None, 0.15), # Stable or near-stable
                fields=fields,
                chunk_size=500
            )
            print(f"Retrieved {len(docs)} materials")
            return docs
        except Exception as e:
            print(f"Error querying Materials Project: {e}")
            return []

docs = get_electrolyte_data()

df = pd.DataFrame([doc.dict() for doc in docs])

print(f"Collected {len(df)} materials")
print(f"Features: {list(df.columns)}")

print("First few rows of the DataFrame:")
print(df.head())
# Save raw data
df.to_csv("nitride_electrolyte_data.csv", index=False)

Querying Materials Project with filters:


Retrieving SummaryDoc documents: 100%|██████████| 3055/3055 [00:02<00:00, 1033.58it/s]


Retrieved 3055 materials
Collected 3055 materials
Features: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'bulk_modulus', 'shear_modulus', 'universal_anisotropy', 